In [87]:
!pip install spotipy

In [88]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

In [89]:
client_id = 'e147f5c0f9bb4479a3f04ed6d9bd90c0'
client_secret= 'dcd9d78261de46e08499ae7fb1ffdd2e'
scope = 'user-library-read'

In [90]:
token = util.prompt_for_user_token(scope, client_id= client_id, client_secret=client_secret, redirect_uri='http://localhost:8881/callback')

In [91]:
sp = spotipy.Spotify(auth=token)

In [92]:
playlist_dic = {}
playlist_cover_art = {}
for i in sp.current_user_playlists()['items']:

    playlist_dic[i['name']] = i['uri'].split(':')[2]
    playlist_cover_art[i['uri'].split(':')[2]] = i['images'][0]['url']

In [93]:
playlist_dic

{'Chill': '37rxHwFCO9C0vHq209EhP2',
 'Hip hop': '3bYlAHuM95WmRPhB7vxpRK',
 'FR': '5USkQd9Temod7pyqUkGnzK',
 'Pop': '53a6KY4Vq4HEctO0D1rZUf'}

In [94]:
def generate_playlist_df(playlist_name, playlist_dic, spotify_data):
    
    playlist = pd.DataFrame()

    for i, j in enumerate(sp.playlist(playlist_dic[playlist_name])['tracks']['items']):
        playlist.loc[i, 'artist'] = j['track']['artists'][0]['name']
        playlist.loc[i, 'track_name'] = j['track']['name']
        playlist.loc[i, 'track_id'] = j['track']['id']
        playlist.loc[i, 'url'] = j['track']['album']['images'][1]['url']
        playlist.loc[i, 'date_added'] = j['added_at']

    playlist['date_added'] = pd.to_datetime(playlist['date_added'])  
    
    playlist = playlist[playlist['track_id'].isin(spotify_data['track_id'].values)].sort_values('date_added',ascending = False)

    return playlist

In [95]:
import pandas as pd
spotify_data = pd.read_csv('SpotifyFeatures.csv')

In [96]:
spotify_data.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.910,0.000,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,137373,0.737,0.000,F#,0.1510,-5.559,Minor,0.0868,174.003,4/4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,170267,0.131,0.000,C,0.1030,-13.879,Minor,0.0362,99.488,5/4,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,152427,0.326,0.000,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.950,0.331,82625,0.225,0.123,F,0.2020,-21.150,Major,0.0456,140.576,4/4,0.390


In [97]:
playlist_df = generate_playlist_df('Chill', playlist_dic, spotify_data)

In [98]:
playlist_df.head()

,artist,track_name,track_id,url,date_added
58,Tyga,Dip (feat. Nicki Minaj),4hXDobvUaXyDFLnmecoY0z,https://i.scdn.co/image/ab67616d00001e02e387cf...,2023-02-09 19:30:11+00:00
57,Gorillaz,She's My Collar (feat. Kali Uchis),3lIxtCaROdRDuTnNBDm3n2,https://i.scdn.co/image/ab67616d00001e02a5952c...,2023-02-07 22:55:31+00:00
56,Tame Impala,Eventually,5M4yti0QxgqJieUYaEXcpw,https://i.scdn.co/image/ab67616d00001e029e1cfc...,2023-02-07 22:53:36+00:00
54,Icona Pop,I Love It (feat. Charli XCX),6HZ9VeI5IRFCNQLXhpF4bq,https://i.scdn.co/image/ab67616d00001e024e2e60...,2023-02-06 11:56:28+00:00
52,Tame Impala,The Less I Know The Better,6K4t31amVTZDgR3sKmwUJJ,https://i.scdn.co/image/ab67616d00001e029e1cfc...,2023-02-06 11:13:30+00:00


In [99]:
from skimage import io
import matplotlib.pyplot as plt

def visualize_cover_art(playlist_df):
    temp = playlist_df['url'].values
    plt.figure(figsize=(15,int(0.625 * len(temp))) , facecolor='#8cfc03')
    columns = 5
    
    for i, url in enumerate(temp):
        plt.subplot(len(temp) / columns + 1, columns, i + 1)

        image = io.imread(url)
        plt.imshow(image)
        plt.xticks([])
        plt.yticks([])
        s='' 
        plt.xlabel(s.join(playlist_df['track_name'].values[i].split(' ')[:4]), fontsize = 10, fontweight='bold')
        plt.tight_layout(h_pad=0.8, w_pad=0)
        plt.subplots_adjust(wspace=None, hspace=None)

    plt.show()

In [100]:
#visualize_cover_art(playlist_df)

In [101]:
playlist_df.to_csv('playlist.csv')

In [102]:
spotify_features_df = spotify_data

In [103]:
genre_OHE = pd.get_dummies(spotify_features_df.genre)
genre_OHE.head()

,A Capella,Alternative,Anime,Blues,Children's Music,Children’s Music,Classical,Comedy,Country,Dance,...,Pop,R&B,Rap,Reggae,Reggaeton,Rock,Ska,Soul,Soundtrack,World
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [104]:
key_OHE = pd.get_dummies(spotify_features_df.key)
key_OHE.head()

,A,A#,B,C,C#,D,D#,E,F,F#,G,G#
0,0,0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,0,0


In [105]:
from sklearn.preprocessing import MinMaxScaler

In [106]:
scaled_features = MinMaxScaler().fit_transform([spotify_features_df['acousticness'].values,
                                                spotify_features_df['danceability'].values,
                                                spotify_features_df['duration_ms'].values,
                                                spotify_features_df['energy'].values,
                                                spotify_features_df['instrumentalness'].values,
                                                spotify_features_df['liveness'].values,
                                                spotify_features_df['loudness'].values,
                                                spotify_features_df['speechiness'].values,
                                                spotify_features_df['tempo'].values,
                                                spotify_features_df['valence'].values])

In [107]:
spotify_features_df[['acousticness','danceability','duration_ms','energy','instrumentalness','liveness','loudness',
                     'speechiness','tempo','valence']] = scaled_features.T

In [108]:
spotify_features_df = spotify_features_df.drop('genre',axis = 1)
spotify_features_df = spotify_features_df.drop('artist_name', axis = 1)
spotify_features_df = spotify_features_df.drop('track_name', axis = 1)
spotify_features_df = spotify_features_df.drop('popularity',axis = 1)
spotify_features_df = spotify_features_df.drop('key', axis = 1)
spotify_features_df = spotify_features_df.drop('mode', axis = 1)
spotify_features_df = spotify_features_df.drop('time_signature', axis = 1)

In [109]:
spotify_features_df = spotify_features_df.join(genre_OHE)
spotify_features_df = spotify_features_df.join(key_OHE)

In [110]:
spotify_features_df.head()

,track_id,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,...,B,C,C#,D,D#,E,F,F#,G,G#
0,0BRjO6ga9RKCKjfDqeFgWV,0.000025,0.000022,1.0,0.000028,0.000018,0.000022,0.0,0.000019,0.001699,...,0,0,1,0,0,0,0,0,0,0
1,0BjC1NfoEOOusryehmNudP,0.000042,0.000045,1.0,0.000046,0.000040,0.000042,0.0,0.000041,0.001307,...,0,0,0,0,0,0,0,1,0,0
2,0CoSDzoNIKCRs124s9uTVy,0.000087,0.000085,1.0,0.000082,0.000082,0.000082,0.0,0.000082,0.000666,...,0,1,0,0,0,0,0,0,0,0
3,0Gc6TVm52BwZD07Ki6tIvf,0.000084,0.000081,1.0,0.000082,0.000080,0.000081,0.0,0.000080,0.001207,...,0,0,1,0,0,0,0,0,0,0
4,0IuslXpMROHdEPvSl1fTQK,0.000267,0.000260,1.0,0.000259,0.000257,0.000258,0.0,0.000256,0.001957,...,0,0,0,0,0,0,1,0,0,0


In [111]:
from datetime import datetime

In [112]:
def generate_playlist_vector(spotify_features, playlist_df, weight_factor):
    
    spotify_features_playlist = spotify_features[spotify_features['track_id'].isin(playlist_df['track_id'].values)]
    spotify_features_playlist = spotify_features_playlist.merge(playlist_df[['track_id','date_added']], on = 'track_id',
                                                                how = 'inner')
    
    spotify_features_nonplaylist = spotify_features[~spotify_features['track_id'].isin(playlist_df['track_id'].values)]
    
    playlist_feature_set = spotify_features_playlist.sort_values('date_added',ascending=False)
    
    
    most_recent_date = playlist_feature_set.iloc[0,-1]
    
    for ix, row in playlist_feature_set.iterrows():
        playlist_feature_set.loc[ix,'days_from_recent'] = int((most_recent_date.to_pydatetime()
                                                               - row.iloc[-1].to_pydatetime()).days)
        
    
    playlist_feature_set['weight'] = playlist_feature_set['days_from_recent'].apply(lambda x: weight_factor ** (-x))
    
    playlist_feature_set_weighted = playlist_feature_set.copy()
    
    playlist_feature_set_weighted.update(playlist_feature_set_weighted.iloc[:,:-3]
                                         .mul(playlist_feature_set_weighted.weight.astype(int),0))   
    
    playlist_feature_set_weighted_final = playlist_feature_set_weighted.iloc[:, :-3]
    

    
    return playlist_feature_set_weighted_final.sum(axis = 0), spotify_features_nonplaylist

In [113]:
playlist_vector, nonplaylist_df = generate_playlist_vector(spotify_features_df, playlist_df, 1.2)

In [114]:
from sklearn.metrics.pairwise import cosine_similarity

In [115]:
def generate_recommendation(spotify_data, playlist_vector, nonplaylist_df):

    non_playlist = spotify_data[spotify_data['track_id'].isin(nonplaylist_df['track_id'].values)]
    non_playlist['sim'] = cosine_similarity(nonplaylist_df.drop(['track_id'], axis = 1).values, playlist_vector.drop(labels = 'track_id').values.reshape(1, -1))[:,0]
    non_playlist_top15 = non_playlist.sort_values('sim',ascending = False).head(15)
    non_playlist_top15['url'] = non_playlist_top15['track_id'].apply(lambda x: sp.track(x)['album']['images'][1]['url'])
    
    return  non_playlist_top15

In [116]:
top15 = generate_recommendation(spotify_data, playlist_vector, nonplaylist_df)  

C:\Users\SBiswas\AppData\Local\Temp/ipykernel_16040/43927750.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_playlist['sim'] = cosine_similarity(nonplaylist_df.drop(['track_id'], axis = 1).values, playlist_vector.drop(labels = 'track_id').values.reshape(1, -1))[:,0]


In [117]:
top15

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,sim,url
70459,Hip-Hop,Lil Pump,Back (feat. Lil Yachty),32zhF8yXc1lej5EcudJXp7,61,0.000041,0.000045,1.0,0.000044,0.000041,D,0.000042,0.0,Major,0.000042,0.000695,4/4,0.000044,0.881917,https://i.scdn.co/image/ab67616d00001e02d52ef1...
118400,Rap,Lil Pump,Back (feat. Lil Yachty),32zhF8yXc1lej5EcudJXp7,61,0.000041,0.000045,1.0,0.000044,0.000041,D,0.000042,0.0,Major,0.000042,0.000695,4/4,0.000044,0.881917,https://i.scdn.co/image/ab67616d00001e02d52ef1...
117811,Rap,Isaiah Rashad,Smile,3UNoPBNWST9mQleI5T53zL,59,0.000043,0.000044,1.0,0.000043,0.000040,D,0.000042,0.0,Major,0.000041,0.000690,4/4,0.000043,0.881917,https://i.scdn.co/image/ab67616d00001e020ecc57...
69439,Hip-Hop,Isaiah Rashad,Smile,3UNoPBNWST9mQleI5T53zL,59,0.000043,0.000044,1.0,0.000043,0.000040,D,0.000042,0.0,Major,0.000041,0.000690,4/4,0.000043,0.881917,https://i.scdn.co/image/ab67616d00001e020ecc57...
92301,Hip-Hop,$uicideBoy$,Marlboros & White Widow,7pu5jnOlS1k4fmigtRaXvG,52,0.000040,0.000044,1.0,0.000043,0.000040,D,0.000041,0.0,Major,0.000041,0.000695,4/4,0.000041,0.881917,https://i.scdn.co/image/ab67616d00001e0275242c...
90621,Hip-Hop,SahBabii,Boyfriend (feat. T3),4LK25Fc1J4sQAGqZZIoc2B,52,0.000041,0.000044,1.0,0.000042,0.000040,D,0.000040,0.0,Major,0.000041,0.000687,4/4,0.000041,0.881917,https://i.scdn.co/image/ab67616d00001e023f202b...
111212,Pop,Rainbow Kitten Surprise,It's Called: Freefall,474uVhyGgK5MtY9gMcDgGl,65,0.000046,0.000047,1.0,0.000045,0.000042,D,0.000046,0.0,Major,0.000042,0.000685,4/4,0.000044,0.881917,https://i.scdn.co/image/ab67616d00001e02952921...
151272,Pop,James Bay,Incomplete,78O8CIzX5ZMuJh39fQOR3i,55,0.000044,0.000043,1.0,0.000043,0.000041,D,0.000041,0.0,Major,0.000041,0.000686,4/4,0.000042,0.881917,https://i.scdn.co/image/ab67616d00001e02b36f5e...
110775,Pop,Rainbow Kitten Surprise,Devil Like Me,0HsGh7cboYz6cVMIyFrQ8j,67,0.000040,0.000040,1.0,0.000040,0.000036,D,0.000037,0.0,Major,0.000037,0.000690,4/4,0.000039,0.881917,https://i.scdn.co/image/ab67616d00001e0217d131...
151877,Pop,Greyson Chance,Seasons,7jE3qTchcLVuyejjIk67GU,57,0.000041,0.000040,1.0,0.000040,0.000038,D,0.000038,0.0,Major,0.000038,0.000685,4/4,0.000038,0.881917,https://i.scdn.co/image/ab67616d00001e02c280b1...


In [118]:
#visualize_cover_art(top15)